In [102]:
import zipfile
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [103]:
zf = zipfile.ZipFile("playground-series-s4e1.zip")
zf.namelist()

['sample_submission.csv', 'test.csv', 'train.csv']

In [104]:
df_train = pd.read_csv(zf.open("train.csv"))
df_test = pd.read_csv(zf.open("test.csv"))

In [105]:
#Checking to see if any rows/columns are missing any data
df_train.head(5)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [106]:
#Checking if any columns have words that need to be converted to integers/floats for prediction calculations

df_train.drop(['Surname', 'id', 'CustomerId'], axis =1, inplace=True)
df_train.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [107]:
df_dummy_train = pd.get_dummies(df_train, drop_first= False, dtype=float)

df_dummy_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,668,33.0,3,0.00,2,1.0,0.0,181449.97,0,1.0,0.0,0.0,0.0,1.0
1,627,33.0,1,0.00,2,1.0,1.0,49503.50,0,1.0,0.0,0.0,0.0,1.0
2,678,40.0,10,0.00,2,1.0,0.0,184866.69,0,1.0,0.0,0.0,0.0,1.0
3,581,34.0,2,148882.54,1,1.0,1.0,84560.88,0,1.0,0.0,0.0,0.0,1.0
4,716,33.0,5,0.00,2,1.0,1.0,15068.83,0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,667,33.0,2,0.00,1,1.0,1.0,131834.75,0,0.0,0.0,1.0,1.0,0.0
165030,792,35.0,3,0.00,1,0.0,0.0,131834.45,0,1.0,0.0,0.0,0.0,1.0
165031,565,31.0,5,0.00,1,1.0,1.0,127429.56,0,1.0,0.0,0.0,0.0,1.0
165032,554,30.0,7,161533.00,1,0.0,1.0,71173.03,0,0.0,0.0,1.0,1.0,0.0


In [108]:
y = df_dummy_train['Exited']
x = df_dummy_train.drop(['Exited'], axis =1)

In [109]:
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y , random_state=42)

In [110]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [111]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( x_train, y_train), ( x_test, y_test)]
    
    clf.fit(x_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(x_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [112]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                 
0.8490995903923992                                     
  1%|          | 1/100 [00:01<02:49,  1.71s/trial, best loss: -0.8490995903923992]

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                            
0.8508446641944788                                                                
  2%|▏         | 2/100 [00:03<02:48,  1.72s/trial, best loss: -0.8508446641944788]

...

SCORE:                                                                             
0.8518626239123586                                                                 
 98%|█████████▊| 98/100 [02:46<00:03,  1.69s/trial, best loss: -0.8518626239123586]

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                             
0.8509900870113187                                                                 
 99%|█████████▉| 99/100 [02:48<00:01,  1.72s/trial, best loss: -0.8518626239123586]

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                             
0.8506265299692188                                                                 
100%|██████████| 100/100 [02:49<00:00,  1.70s/trial, best loss: -0.8518626239123586]


In [113]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'colsample_bytree': 0.890109925715352, 'gamma': 6.280880383614678, 'max_depth': 12.0, 'min_child_weight': 10.0, 'reg_alpha': 49.0, 'reg_lambda': 0.45724982713787765}


In [128]:
model = xgb.XGBClassifier(seed = 42,
                          objective='binary:logistic',
                          gamma = 6.280880383614678,
                          colsample_bytree =0.890109925715352,
                          max_depth = 12,
                          min_child_weight = 10.0,
                          reg_alpha = 49.0,
                          reg_lambda = 0.45724982713787765)


In [129]:
model.fit(x_train, 
          y_train, 
          verbose=True,
          eval_metric='auc',
          eval_set=[(x_test, y_test)])

[0]	validation_0-auc:0.87214
[1]	validation_0-auc:0.88018
[2]	validation_0-auc:0.88141
[3]	validation_0-auc:0.88320


C:\Users\heroa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[4]	validation_0-auc:0.88407
[5]	validation_0-auc:0.88451
[6]	validation_0-auc:0.88482
[7]	validation_0-auc:0.88590
[8]	validation_0-auc:0.88612
[9]	validation_0-auc:0.88645
[10]	validation_0-auc:0.88691
[11]	validation_0-auc:0.88730
[12]	validation_0-auc:0.88730
[13]	validation_0-auc:0.88749
[14]	validation_0-auc:0.88759
[15]	validation_0-auc:0.88768
[16]	validation_0-auc:0.88781
[17]	validation_0-auc:0.88780
[18]	validation_0-auc:0.88780
[19]	validation_0-auc:0.88783
[20]	validation_0-auc:0.88790
[21]	validation_0-auc:0.88791
[22]	validation_0-auc:0.88799
[23]	validation_0-auc:0.88798
[24]	validation_0-auc:0.88805
[25]	validation_0-auc:0.88809
[26]	validation_0-auc:0.88809
[27]	validation_0-auc:0.88809
[28]	validation_0-auc:0.88809
[29]	validation_0-auc:0.88809
[30]	validation_0-auc:0.88809
[31]	validation_0-auc:0.88809
[32]	validation_0-auc:0.88809
[33]	validation_0-auc:0.88809
[34]	validation_0-auc:0.88809
[35]	validation_0-auc:0.88809
[36]	validation_0-auc:0.88809
[37]	validation_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.890109925715352, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=6.280880383614678,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=10.0, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [130]:
y_pred = model.predict(x_test)

In [131]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.87

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     32529
           1       0.75      0.55      0.63      8730

    accuracy                           0.87     41259
   macro avg       0.82      0.75      0.78     41259
weighted avg       0.86      0.87      0.86     41259



In [132]:

df_test.drop(['id', 'CustomerId', 'Surname'], axis = 1, inplace = True)

df_dummy_test = pd.get_dummies(df_test, drop_first= False, dtype=float)
df_dummy_test

KeyError: "['id', 'CustomerId', 'Surname'] not found in axis"

In [133]:
y_test_pred = model.predict(df_dummy_test)
y_test_prob = model.predict_proba(df_dummy_test)[:, 1]

y_test_pred
y_test_prob = y_test_prob.round(1)




In [134]:
index = pd.read_csv(zf.open("test.csv"))

In [135]:
df_submission = pd.DataFrame({
    'id': index['id'],
    'Exited': y_test_prob
})
df_submission

,id,Exited
0,165034,0.0
1,165035,0.8
2,165036,0.0
3,165037,0.2
4,165038,0.4
...,...,...
110018,275052,0.0
110019,275053,0.1
110020,275054,0.0
110021,275055,0.1


In [136]:
df_submission.to_csv('XGBoost_Submission_hypertuned_2.csv', index=False)